In [1]:
from google_apis import create_service

CLIENT_FILE = 'client_secret.json'
API_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://mail.google.com/']

service = create_service(CLIENT_FILE, API_NAME, API_VERSION, SCOPES)

gmail v1 service created successfully


In [2]:
def search_emails(query_string, label_ids=None):
    try:
        message_list_response = service.users().messages().list(
                userId='me',
                labelIds=label_ids,
                q=query_string
        ).execute()
        message_items = message_list_response.get('messages')
        next_page_token = message_list_response.get('nextPageToken')
        return message_items
    except Exception as e:
        print(e)
        
def get_file_data(message_id, attachment_id, file_name, save_locatioin):
    response = service.users().messages().attachments().get(
        userId='me',
        messageId=message_id,
        id=attachment_id
    ).execute()
    
    file_data = base64.urlsafe_b64decode(response.get('data').encode('utf-8'))
    return file_data

def get_message_detail(message_id, msg_format='metadata', metadata_headers=None):
    message_detail = service.users().messages().get(
        userId='me',
        id=message_id,
        format=msg_format,
        metadataHeaders=metadata_headers
    ).execute()
    return message_detail

In [23]:
import os
import base64
from tqdm import tqdm

def download_excel():
    query_string = 'has:attachment'
    save_folder = os.path.join(os.path.dirname(os.getcwd()), 'fugle')
    email_messages = search_emails(query_string)

    for email_message in tqdm(email_messages):
        message_detail = get_message_detail(email_message['id'], msg_format='full', metadata_headers=['parts'])
        if '富果帳戶 交易明細' in message_detail['snippet'] and not 'Forwarded' in message_detail['snippet']:
            message_detail_payload = message_detail.get('payload')
            if 'parts' in message_detail_payload:
                for msg_payload in message_detail_payload['parts']:
                    file_name = msg_payload['filename']
                    body = msg_payload['body']
                    if 'attachmentId' in body:
                        folder = file_name[4:10]
                        if file_name == '交易明細20220928.xlsx':
                            return
                        attachment_id = body['attachmentId']
                        attachment_content = get_file_data(email_message['id'], attachment_id, file_name, save_folder)
                        with open(os.path.join(save_folder, folder, file_name), 'wb') as f:
                            f.write(attachment_content)
                            print(f"File {file_name} is saved at {save_folder} {folder}")
download_excel()

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]


In [13]:
# import os
# from datetime import datetime
# this_year = str(datetime.now().year)

# for i in range(1,13):
#     folder_path = os.path.join(save_folder, this_year + str(i).zfill(2))
#     if not os.path.exists(folder_path):
#         os.mkdir(folder_path)
#         print(folder_path)

D:\Mia\Stock\fugle\202202
D:\Mia\Stock\fugle\202203
D:\Mia\Stock\fugle\202204
D:\Mia\Stock\fugle\202205
D:\Mia\Stock\fugle\202206
D:\Mia\Stock\fugle\202207
D:\Mia\Stock\fugle\202208
D:\Mia\Stock\fugle\202209
D:\Mia\Stock\fugle\202210
D:\Mia\Stock\fugle\202211
D:\Mia\Stock\fugle\202212


'2022'